In [15]:
import json
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import joblib

In [16]:
# Load the saved models
svm_model = joblib.load('svm_model.pkl')
pca = joblib.load('pca_model.pkl')

# Load the base model for feature extraction
from tensorflow.keras.applications import ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Load class indices
with open('class_indices.json', 'r') as f:
    class_indices = json.load(f)

# Reverse class mapping and ensure integer keys
class_indices = {int(v): k for k, v in class_indices.items()}

In [17]:

# Function to predict a single image
def predict_single_image(image_path):
    # Step 1: Load and preprocess the image
    image = load_img(image_path, target_size=(128, 128))  # Resize to match ResNet50 input
    image_array = img_to_array(image) / 255.0            # Normalize pixel values
    image_array = np.expand_dims(image_array, axis=0)    # Add batch dimension

    # Step 2: Extract features using ResNet50
    print("Extracting features for the image...")
    features = base_model.predict(image_array, verbose=0)
    features_flattened = features.reshape(1, -1)  # Flatten the features

    # Step 3: Apply PCA for dimensionality reduction
    features_reduced = pca.transform(features_flattened)

    # Step 4: Make prediction with SVM
    prediction = svm_model.predict(features_reduced)
    probabilities = svm_model.predict_proba(features_reduced)

    # Step 5: Map prediction to class label
    predicted_label = class_indices[prediction[0]]  # prediction[0] is an integer
    confidence = probabilities[0][prediction[0]] * 100

    return predicted_label, confidence




In [24]:
# Example usage
image_path = r"S:\Datasets\Datasets\Animal Plant Dataset\New Plant Diseases Dataset(Augmented)\Train\Tomato___Early_blightttttttttttttttttttttttttttttt\Tomato___Early_blightttttttttttttttttttttttttttttt_356.JPG"  # Replace with your image path
predicted_label, confidence = predict_single_image(image_path)
print(f"Predicted Label: {predicted_label}, Confidence: {confidence:.2f}%")

Extracting features for the image...
Predicted Label: Tomato___Early_blightttttttttttttttttttttttttttttt, Confidence: 28.52%
